In [6]:
import google.auth
from google.cloud import bigquery

## Get the project_id
CREDENTIALS, PROJECT_ID = google.auth.default()
print(f"Detected Project ID: {PROJECT_ID}")

## Set various names
DATASET_ID = "bootcamp_challenge3"
TRAINING_DATA = "weather_data"
MODEL_NAME = "weather_forecast"
FINAL_DATA = "weather_reports_final"

GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv"

## Connect BigQuery Client and create dataset

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"

try:
    client.create_dataset(dataset_ref)
    print("Dataset created.")
except Exception:
    print("Dataset already exists.")

## Ingeset data
table_id = f"{PROJECT_ID}.{DATASET_ID}.{TRAINING_DATA}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

load_job = client.load_table_from_uri(
    GCS_URI,
    table_id,
    job_config=job_config,
)

load_job.result()
print("Raw data loaded into BigQuery.")

Detected Project ID: qwiklabs-gcp-00-c2e92c8fc9eb
Dataset already exists.
Raw data loaded into BigQuery.


In [7]:
## Load data from BigQuery

import pandas as pd

query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{TRAINING_DATA}`
"""

df = client.query(query).to_dataframe()
df.head()

,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,2025-03-15,Atlanta,GA,15.1,17.0,0.93,29.57,64.9,Snowy
1,2025-03-15,Atlanta,GA,15.1,17.0,0.93,29.57,64.9,Snowy
2,2025-03-15,Atlanta,GA,15.1,17.0,0.93,29.57,64.9,Snowy
3,2025-03-15,Atlanta,GA,15.1,17.0,0.93,29.57,64.9,Snowy
4,2025-03-15,Atlanta,GA,15.1,17.0,0.93,29.57,64.9,Snowy


In [8]:
df.columns

Index(['date', 'city', 'state', 'temperature_f', 'wind_speed_mph',
       'precipitation_in', 'barometric_pressure_inHg', 'humidity_percent',
       'weather_condition'],
      dtype='object')

In [9]:
## Set up LLM connection
query = f"""
CREATE OR REPLACE MODEL `{DATASET_ID}.{MODEL_NAME}`
  REMOTE WITH CONNECTION DEFAULT
  OPTIONS(ENDPOINT = 'gemini-2.5-flash');
"""

print(query)
client.query(query).result()


CREATE OR REPLACE MODEL `bootcamp_challenge3.weather_forecast`
  REMOTE WITH CONNECTION DEFAULT
  OPTIONS(ENDPOINT = 'gemini-2.5-flash');



In [10]:
## Construct the prompt and generate reports
report_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{FINAL_DATA}` AS
SELECT
  date,
  city,
  state,
  temperature_f,
  wind_speed_mph,
  precipitation_in,
  barometric_pressure_inHg,
  humidity_percent,
  weather_condition,
  prompt,
  JSON_VALUE(ml_generate_text_result, '$.candidates[0].content.parts[0].text') AS weather_report
FROM
  ML.GENERATE_TEXT(
    MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
    (
      SELECT
        *,
        CONCAT(
          "Generate a local weather update for ", city, ". ",
          "Today is ",date,", ",
          "The current temperature is ", temperature_f, " degrees with ",
          humidity_percent, "humidity. The sky is currently showing ", weather_condition, ". ",
          "Based on these specific conditions, provide a 'feel' description (e.g., muggy, crisp, or dry) ",
          "and suggest one specific activity suitable for this weather."
        ) AS prompt
      FROM
        `{PROJECT_ID}.{DATASET_ID}.{TRAINING_DATA}`
    ),
    STRUCT(
      0.2 AS temperature,
      1024 AS max_output_tokens,
      0.8 AS top_p,
      20 AS top_k
    )
  );
"""

print("Generating reports and saving to new table...")
query_job = client.query(report_query)
query_job.result()

print(f"Success! View your reports in table: {DATASET_ID}.{FINAL_DATA}")

Generating reports and saving to new table...
Success! View your reports in table: bootcamp_challenge3.weather_reports_final


In [11]:
## Print results for review
results_query = f"SELECT * FROM `{DATASET_ID}.{FINAL_DATA}`"
df_results = client.query(results_query).to_dataframe()

df_results[["weather_report"]].head(10)

,weather_report
0,"Good morning, Atlanta! On this Friday, Februar..."
1,"Good morning, Atlanta!\n\nCurrently, it's **55..."
2,Here's your local weather update for Atlanta o...
3,"Good morning, Atlanta! On this Friday, Februar..."
4,Here's your local weather update for Atlanta:\...
5,Here's your local weather update for Atlanta:\...
6,Here's your local weather update for Atlanta o...
7,"Good morning, Atlanta! On this Wednesday, Febr..."
8,Here's your local weather update for Atlanta o...
9,Here's your local weather update for Atlanta:\...
